In [1]:
pip install pytorch-lightning pytorch-forecasting


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Hourly price

data_2023 = [pd.read_csv(f"Data/messari_hourly_prices/hourly_prices_2023_{i}.csv", parse_dates=["datetime"]) for i in range(1, 13)]
data_2024 = [pd.read_csv(f"Data/messari_hourly_prices/hourly_prices_2024_{i}.csv", parse_dates=["datetime"]) for i in range(1, 13)]
df1 = pd.concat(data_2023 + data_2024).sort_values("datetime").set_index("datetime").drop(columns=["Unnamed: 0"])

df1 = df1.reset_index()
df1_btc = df1[df1['reserve_name']=='Wrapped BTC']

# Reserves features

data2_2023 = [pd.read_csv(f"Data/reserves_features/reserves_history_hourly_completed_2023-{i}.csv") for i in range(1, 13)]
data2_2024 = [pd.read_csv(f"Data/reserves_features/reserves_history_hourly_completed_2024-{i}.csv") for i in range(1, 13)]
dfr = pd.concat(data2_2023 + data2_2024).drop(columns=["Unnamed: 0"])

dfr_btc = dfr[dfr['reserve_name']=='Wrapped BTC'].dropna().drop(["totalPrincipalStableDebt"], axis=1)

# Merge

dfr_btc['regular_datetime'] = pd.to_datetime(dfr_btc['regular_datetime'], utc=True)
df_btc = pd.merge(df1_btc, dfr_btc, left_on='datetime', right_on='regular_datetime')
df_btc = df_btc.drop(['blockNumber', 'timestamp_hours', 'id', 'snapshot_timestamp', 
'regular_datetime', 'reserve_name_x', 'reserve_name_y', 'reserve_pool', 'protocol', 'protocol_name', 
'priceInUsd', 'outputTokenPriceUSD', 'averageStableBorrowRate', 'priceInEth', 'reserve_decimals', 
'true_value', 'timestamp'], axis=1)

df_btc = df_btc.sort_values("datetime")
df_btc

,datetime,inputTokenPriceUSD,accruedToTreasury,availableLiquidity,liquidityIndex,liquidityRate,stableBorrowRate,totalATokenSupply,totalCurrentVariableDebt,totalLiquidity,totalScaledVariableDebt,utilizationRate,variableBorrowIndex,variableBorrowRate
0,2023-01-27 12:00:00+00:00,22901.415798,0.000000,0.619864,1.000000,0.000000,0.09,0.619864,0.000000,0.619864,0.000000,0.000000,1.000000,0.000000
1,2023-01-27 13:00:00+00:00,22889.291609,0.000000,0.768670,1.000000,0.000000,0.09,0.768670,0.000000,0.768670,0.000000,0.000000,1.000000,0.000000
2,2023-01-27 14:00:00+00:00,22840.819903,0.000000,3.443268,1.000000,0.000000,0.09,3.443268,0.000000,3.443268,0.000000,0.000000,1.000000,0.000000
3,2023-01-27 15:00:00+00:00,22979.545483,0.000000,5.263297,1.000000,0.000000,0.09,5.263297,0.000000,5.263297,0.000000,0.000000,1.000000,0.000000
4,2023-01-27 16:00:00+00:00,23008.788367,0.000000,5.318675,1.000000,0.000000,0.09,5.318675,0.000000,5.318675,0.000000,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14018,2024-12-31 19:00:00+00:00,93759.525729,0.036288,33788.341342,1.003206,0.000145,0.00,36595.717590,2787.966074,36511.314003,2730.518880,0.074579,1.021039,0.003809
14019,2024-12-31 20:00:00+00:00,93759.525729,0.036469,33774.009561,1.003206,0.000146,0.00,36592.599123,2799.180682,36508.195222,2741.500825,0.074892,1.021040,0.003825
14020,2024-12-31 21:00:00+00:00,93271.901873,0.037136,33746.943636,1.003206,0.000146,0.00,36583.706269,2817.354629,36499.301831,2759.298887,0.075409,1.021040,0.003823
14021,2024-12-31 22:00:00+00:00,93512.722731,0.038120,33746.963060,1.003206,0.000148,0.00,36583.727180,2817.356429,36499.321755,2759.299377,0.075408,1.021041,0.003851


In [ ]:
import pytorch_forecasting as ptf
import pytorch_lightning as pl

from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data.examples import get_stallion_data
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader

# df_btc['series_id'] = 'BTC'
# df_btc["datetime"] = df_btc.index.astype("int")

# # Define dataset
# training = ptf.TimeSeriesDataSet(
#     df_btc,
#     time_idx='datetime',          # Timestamp column
#     target='inputTokenPriceUSD',           # Target variable
#     group_ids = ['series_id'], 
#     # static_categoricals=[],       # Metadata (e.g., asset type)
#     time_varying_known_reals=[
#         'accruedToTreasury','availableLiquidity', 'liquidityIndex', 'liquidityRate','stableBorrowRate', 
#         'totalATokenSupply','totalCurrentVariableDebt', 'totalLiquidity', 'totalScaledVariableDebt',
#         'utilizationRate', 'variableBorrowIndex', 'variableBorrowRate'
#     ],
#     time_varying_unknown_reals=['inputTokenPriceUSD'],
#     max_encoder_length=24,        # Lookback window (e.g., 24 hours)
#     max_prediction_length=12,     # Forecast horizon (e.g., 12 hours)
# )

In [4]:
# model = TemporalFusionTransformer.from_dataset(training)

# trainer = Trainer(max_epochs=20)
# train_dataloader = training.to_dataloader(train=True, batch_size=64, num_workers=4)
# trainer.fit(model, train_dataloader)


# # Initialize
# tft = TemporalFusionTransformer.from_dataset(
#     training,
#     hidden_size=32,               # Model capacity
#     lstm_layers=2,                # Depth of LSTM
#     attention_head_size=4,        # Multi-head attention
#     dropout=0.1,
# )

# # Train
# trainer = pl.Trainer(max_epochs=10)
# trainer.fit(tft, datamodule=training.to_dataloader(train=True, batch_size=64))

In [ ]:
import numpy as np
import torch

from pytorch_forecasting.metrics import QuantileLoss


# 1. Load data
data = get_stallion_data()

data["date"] = pd.to_datetime(np.array(data["date"]))
data["month"] = data["date"].dt.month
data["year"] = data["date"].dt.year
data["time_idx"] = data["month"] + 12 * (data["year"] - data["year"].min())


# 2. Create dataset
max_encoder_length = 36
max_prediction_length = 6

training = TimeSeriesDataSet(
    data[data.time_idx <= 36 * 3],
    time_idx="time_idx",
    target="volume",
    group_ids=["agency", "sku"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_known_reals=["time_idx", "price_regular", "price_actual"],
    time_varying_unknown_reals=["volume"],
    static_categoricals=["agency", "sku"],
)

# 3. Create dataloaders
train_dataloader = training.to_dataloader(train=True, batch_size=64, num_workers=0)
val_dataloader = training.to_dataloader(train=False, batch_size=64, num_workers=0)

# 4. Create model — THIS is where your error likely came from
model = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

# # 5. Fit with Lightning Trainer
# # trainer = Trainer(max_epochs=20, gradient_clip_val=0.1)
# # trainer.fit(model, train_dataloader, val_dataloader)

# import pytorch_lightning as pl
# from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet

# # Define a LightningModule wrapper for TFT
# class TFTModel(pl.LightningModule):
#     def __init__(self, tft):
#         super().__init__()
#         self.tft = tft

#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self.tft(x)
#         loss = self.tft.loss(y_hat, y)
#         self.log("train_loss", loss)
#         return loss

#     def configure_optimizers(self):
#         return self.tft.configure_optimizers()

# # Initialize TFT
# tft = TemporalFusionTransformer.from_dataset(
#     training,
#     hidden_size=32,
#     lstm_layers=2,
#     attention_head_size=4,
#     dropout=0.1,
# )

# # Wrap TFT in LightningModule
# model = TFTModel(tft)

# # Train with PyTorch Lightning
# trainer = pl.Trainer(max_epochs=10)
# trainer.fit(model, datamodule=training.to_dataloader(train=True, batch_size=64))

/usr/local/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.12/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.


AttributeError: 'DataLoader' object has no attribute 'prepare_data_per_node'

In [15]:
from pytorch_lightning.core.datamodule import LightningDataModule

# 1. Data Module
class TFTDataModule(LightningDataModule):
    def __init__(self, train_ds, val_ds, batch_size=64):
        super().__init__()
        self.train_ds = train_ds
        self.val_ds = val_ds
        self.batch_size = batch_size
        
    def train_dataloader(self):
        return self.train_ds.to_dataloader(train=True, batch_size=self.batch_size)
    
    def val_dataloader(self):
        return self.val_ds.to_dataloader(train=False, batch_size=self.batch_size)

# 2. Model Wrapper (as before)
class TFTModel(pl.LightningModule):
    def __init__(self, tft):
        super().__init__()
        self.tft = tft
        self.save_hyperparameters(ignore=['tft'])

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.tft(x)
        loss = self.tft.loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.tft(x)
        loss = self.tft.loss(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return self.tft.configure_optimizers()

# 3. Initialize and train
validation = TimeSeriesDataSet.from_dataset(
    training,
    validation_data,
    predict=True,
    stop_randomization=True
)

datamodule = TFTDataModule(training, validation)
model = TFTModel(tft)
trainer.fit(model, datamodule=datamodule)

NameError: name 'validation_data' is not defined